In [2]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

In [4]:
df = pd.read_json(r"C:\sample_intern\user-wallet-transactions.json")

In [6]:
df.head()

,_id,userWallet,network,protocol,txHash,logId,timestamp,blockNumber,action,actionData,__v,createdAt,updatedAt
0,{'$oid': '681d38fed63812d4655f571a'},0x00000000001accfa9cef68cf5371a23025b6d4b6,polygon,aave_v2,0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6...,0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6...,2021-08-17 05:29:26,1629178166,deposit,"{'type': 'Deposit', 'amount': '2000000000', 'a...",0,{'$date': '2025-05-08T23:06:39.465Z'},{'$date': '2025-05-08T23:06:39.465Z'}
1,{'$oid': '681aa70dd6df53021cc6f3c0'},0x000000000051d07a4fb3bd10121a343d85818da6,polygon,aave_v2,0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d...,0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d...,2021-05-20 15:36:53,1621525013,deposit,"{'type': 'Deposit', 'amount': '145000000000000...",0,{'$date': '2025-05-07T00:19:26.159Z'},{'$date': '2025-05-07T00:19:26.159Z'}
2,{'$oid': '681d04c2d63812d4654c733e'},0x000000000096026fb41fc39f9875d164bd82e2dc,polygon,aave_v2,0xe2d7eb815c89331a734ed6f204a06c385a1b39040baa...,0xe2d7eb815c89331a734ed6f204a06c385a1b39040baa...,2021-07-24 09:28:33,1627118913,deposit,"{'type': 'Deposit', 'amount': '100000000000000...",0,{'$date': '2025-05-08T19:23:47.877Z'},{'$date': '2025-05-08T19:23:47.877Z'}
3,{'$oid': '681d133bd63812d46551b6ef'},0x000000000096026fb41fc39f9875d164bd82e2dc,polygon,aave_v2,0x0d63a2eacd82b82f868db825ea7385e6bd8d046ee729...,0x0d63a2eacd82b82f868db825ea7385e6bd8d046ee729...,2021-07-31 23:15:18,1627773318,deposit,"{'type': 'Deposit', 'amount': '400000000000000...",0,{'$date': '2025-05-08T20:25:33.141Z'},{'$date': '2025-05-08T20:25:33.141Z'}
4,{'$oid': '681899e4ba49fc91cf2f4454'},0x0000000000e189dd664b9ab08a33c4839953852c,polygon,aave_v2,0x590eabb812c5006a6f4766f44e6e9d3ad0b5b563de69...,0x590eabb812c5006a6f4766f44e6e9d3ad0b5b563de69...,2021-04-19 15:25:07,1618845907,redeemunderlying,"{'type': 'RedeemUnderlying', 'amount': '501548...",0,{'$date': '2025-05-05T10:58:45.934Z'},{'$date': '2025-05-05T10:58:45.934Z'}


In [11]:
df['amount'] = df['actionData'].apply(lambda x: x.get('amount', None))
df['assetPriceUSD'] = df['actionData'].apply(lambda x: x.get('assetPriceUSD', None))

df['amount_usd'] = pd.to_numeric(df['amount'], errors='coerce') / 1e18 \
                       * pd.to_numeric(df['assetPriceUSD'], errors='coerce')

df.dropna(subset=['amount_usd'], inplace=True)


In [12]:
df.head()

,_id,userWallet,network,protocol,txHash,logId,timestamp,blockNumber,action,actionData,__v,createdAt,updatedAt,amount,assetPriceUSD,amount_usd
0,{'$oid': '681d38fed63812d4655f571a'},0x00000000001accfa9cef68cf5371a23025b6d4b6,polygon,aave_v2,0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6...,0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6...,2021-08-17 05:29:26,1629178166,deposit,"{'type': 'Deposit', 'amount': '2000000000', 'a...",0,{'$date': '2025-05-08T23:06:39.465Z'},{'$date': '2025-05-08T23:06:39.465Z'},2000000000,0.9938318274296357543568636362026045,1.987664e-09
1,{'$oid': '681aa70dd6df53021cc6f3c0'},0x000000000051d07a4fb3bd10121a343d85818da6,polygon,aave_v2,0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d...,0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d...,2021-05-20 15:36:53,1621525013,deposit,"{'type': 'Deposit', 'amount': '145000000000000...",0,{'$date': '2025-05-07T00:19:26.159Z'},{'$date': '2025-05-07T00:19:26.159Z'},145000000000000000000,1.970306761113742502077627085754506,2.856945e+02
2,{'$oid': '681d04c2d63812d4654c733e'},0x000000000096026fb41fc39f9875d164bd82e2dc,polygon,aave_v2,0xe2d7eb815c89331a734ed6f204a06c385a1b39040baa...,0xe2d7eb815c89331a734ed6f204a06c385a1b39040baa...,2021-07-24 09:28:33,1627118913,deposit,"{'type': 'Deposit', 'amount': '100000000000000...",0,{'$date': '2025-05-08T19:23:47.877Z'},{'$date': '2025-05-08T19:23:47.877Z'},1000000000000000,0.9223772540040778087468127454060907,9.223773e-04
3,{'$oid': '681d133bd63812d46551b6ef'},0x000000000096026fb41fc39f9875d164bd82e2dc,polygon,aave_v2,0x0d63a2eacd82b82f868db825ea7385e6bd8d046ee729...,0x0d63a2eacd82b82f868db825ea7385e6bd8d046ee729...,2021-07-31 23:15:18,1627773318,deposit,"{'type': 'Deposit', 'amount': '400000000000000...",0,{'$date': '2025-05-08T20:25:33.141Z'},{'$date': '2025-05-08T20:25:33.141Z'},4000000000000000,1.057483342538527030870377652200841,4.229933e-03
4,{'$oid': '681899e4ba49fc91cf2f4454'},0x0000000000e189dd664b9ab08a33c4839953852c,polygon,aave_v2,0x590eabb812c5006a6f4766f44e6e9d3ad0b5b563de69...,0x590eabb812c5006a6f4766f44e6e9d3ad0b5b563de69...,2021-04-19 15:25:07,1618845907,redeemunderlying,"{'type': 'RedeemUnderlying', 'amount': '501548...",0,{'$date': '2025-05-05T10:58:45.934Z'},{'$date': '2025-05-05T10:58:45.934Z'},501548,1.00212969929885247748335868168494,5.026161e-13


In [13]:
wallets = df.groupby('userWallet')


In [17]:
features = pd.DataFrame(index=wallets.groups.keys())
features['wallet_age_days'] = wallets['timestamp'].agg(lambda x: (x.max() - x.min()).total_seconds() / (60 * 60 * 24))
features['tx_count'] = wallets['txHash'].nunique()
features['tx_frequency'] = features['tx_count'] / (features['wallet_age_days'] + 1) # +1 to avoid division by zero
features['total_deposited_usd'] = wallets.apply(lambda x: x[x['action'] == 'deposit']['amount_usd'].sum())
features['total_borrowed_usd'] = wallets.apply(lambda x: x[x['action'] == 'borrow']['amount_usd'].sum())
features['total_repaid_usd'] = wallets.apply(lambda x: x[x['action'] == 'repay']['amount_usd'].sum())
features['liquidation_count'] = wallets.apply(lambda x: x[x['action'] == 'liquidationcall'].shape[0])
features['borrow_to_deposit_ratio'] = features['total_borrowed_usd'] / (features['total_deposited_usd'] + 1e-6)
features['repayment_ratio'] = features['total_repaid_usd'] / (features['total_borrowed_usd'] + 1e-6)

C:\Users\jk913\AppData\Local\Temp\ipykernel_33436\1287586270.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features['total_deposited_usd'] = wallets.apply(lambda x: x[x['action'] == 'deposit']['amount_usd'].sum())
C:\Users\jk913\AppData\Local\Temp\ipykernel_33436\1287586270.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features['total_borrowed_usd'] = wallets.apply(lambda x: x[x['action'] == '

In [20]:
from sklearn.preprocessing import MinMaxScaler

scores = pd.DataFrame(index=features.index)
scaler = MinMaxScaler()

# --- Component Score Calculation ---
scores['css'] = scaler.fit_transform(features[['total_deposited_usd', 'wallet_age_days']]).mean(axis=1)

rbs_features = pd.DataFrame(index=features.index)
rbs_features['repayment'] = features['repayment_ratio']
rbs_features['health'] = 1 - features['borrow_to_deposit_ratio']
scores['rbs'] = scaler.fit_transform(rbs_features).mean(axis=1)

scores['ras'] = 1.0
scores.loc[features['liquidation_count'] > 0, 'ras'] = 0

scores['pes'] = scaler.fit_transform(features[['tx_frequency', 'tx_count']]).mean(axis=1)

# --- Final Weighted Score ---
weights = {'ras': 0.40, 'rbs': 0.30, 'css': 0.20, 'pes': 0.10}
rule_based_score = (scores['ras'] * weights['ras'] + scores['rbs'] * weights['rbs'] +
                    scores['css'] * weights['css'] + scores['pes'] * weights['pes'])

(rule_based_score * 1000).astype(int)


0x00000000001accfa9cef68cf5371a23025b6d4b6    550
0x000000000051d07a4fb3bd10121a343d85818da6    550
0x000000000096026fb41fc39f9875d164bd82e2dc    554
0x0000000000e189dd664b9ab08a33c4839953852c    634
0x0000000002032370b971dabd36d72f3e5a7bf1ee    638
                                             ... 
0x06185db89f5c8ef5e5a0abc95c5cb69b54c33f30    550
0x06189341e523a52ae10c4bd4a7774371fac1b249    551
0x0618c450370822c5cb25ec143a3008230d8e2c12    550
0x0618e615c8c9b5efc8c8eadc68be7182bdc455e2    570
0x06192f889f17bf2aff238d08d8c26cbcfcc7b45a    590
Length: 3497, dtype: int32

In [36]:
from sklearn.model_selection import GridSearchCV

y = rule_based_score.values

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_regressor = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)

grid_search = GridSearchCV(
    estimator=xgb_regressor,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1
)

grid_search.fit(features.values, y)
best_xgb = grid_search.best_estimator_

X_train, X_test, y_train, y_test = train_test_split(features.values, y, test_size=0.2, random_state=42)

# Fit the best model and predict
best_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
predicted_scores = best_xgb.predict(features.values)

scaler = MinMaxScaler(feature_range=(0, 1000))
final_scores = scaler.fit_transform(predicted_scores.reshape(-1, 1)).astype(int)
final_scores = final_scores.flatten()


Fitting 3 folds for each of 108 candidates, totalling 324 fits


In [40]:
best_xgb

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.2, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=-1, num_parallel_tree=None, ...)

In [37]:
final_scores = pd.Series(final_scores, index=features.index, name='final_score')
final_scores = final_scores.sort_values(ascending=False)

In [38]:
final_scores

0x01615d8d53eb9aafcda15853286fdf8259368a68    1000
0x02eca8cc78b7d30c1ac5e16988ed2c8a9da658d6     984
0x0034baeeb160a5f1032b6d124d3e87cc94d74e62     942
0x005f16f017aa933bb41965b52848ceb8ee48b171     915
0x03b16ab6e23bdbeeab719d8e4c49d63674876253     906
                                              ... 
0x02ccbf14d05af1bba1c85c0e4ebe34450b4bc3a1       2
0x037343879ca2c1d0078038549ee042046f32243f       2
0x052a44f7e4213413b9b9440ded0175a3b43bffc1       0
0x06089bceee94a485c61b311cb61934d5cf8a684e       0
0x0097f65bdc45149e5ffac00e4c51a5b073af7245       0
Name: final_score, Length: 3497, dtype: int32